In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv wandb
!pip install -q namex
!apt install python3.10-venv
!git clone https://github.com/keras-team/keras-core.git && cd keras-core
!python pip_build.py --install

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import tensorflow_datasets as tfds

import keras_cv
from keras_cv import visualization

import keras_core as keras
from keras_core import ops

import numpy as np

import resource
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import wandb
from wandb_addons.keras.detection import visualize_dataset

In [ ]:
wandb.init(project="keras-cv-callbacks", job_type="detection")

config = wandb.config
config.batch_size = 4

In [ ]:
class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
config.class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [ ]:
def unpackage_raw_tfds_inputs(inputs, bounding_box_format):
    image = inputs["image"]
    boxes = keras_cv.bounding_box.convert_format(
        inputs["objects"]["bbox"],
        images=image,
        source="rel_yxyx",
        target=bounding_box_format,
    )
    bounding_boxes = {
        "classes": tf.cast(inputs["objects"]["label"], dtype=tf.float32),
        "boxes": tf.cast(boxes, dtype=tf.float32),
    }
    return {
        "images": tf.cast(image, tf.float32),
        "bounding_boxes": bounding_boxes
    }


def load_pascal_voc(split, dataset, bounding_box_format):
    ds = tfds.load(dataset, split=split, with_info=False, shuffle_files=True)
    ds = ds.map(
        lambda x: unpackage_raw_tfds_inputs(
            x, bounding_box_format=bounding_box_format
        ),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    return ds


train_ds = load_pascal_voc(
    split="train", dataset="voc/2007", bounding_box_format="xywh"
)
eval_ds = load_pascal_voc(split="test", dataset="voc/2007", bounding_box_format="xywh")

train_ds = train_ds.shuffle(config.batch_size * 4)

In [ ]:
train_ds = train_ds.ragged_batch(config.batch_size, drop_remainder=True)
eval_ds = eval_ds.ragged_batch(config.batch_size, drop_remainder=True)

In [ ]:
visualize_dataset(
    dataset=train_ds,
    class_mapping=config.class_mapping,
    title="Train-Dataset",
    max_batches_to_visualize=2
)
visualize_dataset(
    dataset=eval_ds,
    class_mapping=config.class_mapping,
    title="Eval-Dataset",
    max_batches_to_visualize=2
)

In [ ]:
wandb.finish()